<a href="https://colab.research.google.com/github/rafaelpuyau/PUC-Rio/blob/main/MVP_Sprint_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

Previsão dos preços dos carros da tabela FIPE

## Metadados

* __year_of_reference__: year of reference of the observation, i.e., the year the data corresponds to.

* __moth_of_reference__: month of reference of the observation, i.e., the month the data corresponds to. The average price is calculated by FIPE each month.

* __fipe_code__: unique id corresponding to a model for easy search on FIPE webpage.

* __authentication__: unique code that authenticates the consult in FIPE's site.
brand: car's make.

* __model__: a description of the car containing the name and other descriptive information, as provided in FIPE table.

* __fuel__: fuel used by the car. Some of gas cars are actually alcohol and gas (totalflex), which is common in Brazil.

* __gear__: the way gears are shifted.

* __engine_size__: Engine size measured in cubic centimeters.

* __year_model__: those values corresponds to the year of reference, and may not be the same of the year of manufacture, which in case will corresponds to a year before year_model. Observations with year_model = year_of_reference mean the car is brand new for that year of reference, i.e., a 2021 car with year_of_reference = 2021 and moth_of_reference = July mean that the observation (mainly the average price) corresponds to a brand new car in the year of 2021, of the month of July. The same model may have a different average price for different month.

* __avg_price_brl__: average car's price, as measured by FIPE, in BRL

In [ ]:
# Distribuição das variáveis categóricas
# for col in ['month_of_reference', 'brand', 'fuel', 'gear', 'year_model']:
#   print(df[col].value_counts())
#   print('---' * 20)

Uma breve e simples __EDA__ (Exploratory Data Analysis) para verificar possíveis correlações e entender melhor a base

In [138]:
print(df.describe())

,year_of_reference,engine_size,year_model,avg_price_brl
count,599007.00,599007.00,599007.00,599007.00
mean,2021.57,2.27,2008.75,111580.60
std,0.57,1.03,9.19,291223.74
min,2021.00,0.70,1985.00,1679.00
25%,2021.00,1.60,2001.00,20150.00
50%,2022.00,2.00,2010.00,42989.00
75%,2022.00,2.80,2016.00,96921.00
max,2023.00,6.70,2023.00,8600000.00


## Separando variáveis preditoras e target

In [35]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder # encoder para transformar colunas categóricas em numéricas
from sklearn.model_selection import train_test_split # para particionar em bases de treino e teste (holdout)
from sklearn.model_selection import KFold # para preparar os folds da validação cruzada
from sklearn.model_selection import cross_val_score # para executar a validação cruzada
from sklearn.metrics import mean_squared_error # métrica de avaliação MSE
from sklearn.linear_model import LinearRegression # algoritmo Regressão Linear
from sklearn.linear_model import Ridge # algoritmo Regularização Ridge
from sklearn.linear_model import Lasso # algoritmo Regularização Lasso
from sklearn.neighbors import KNeighborsRegressor # algoritmo KNN
from sklearn.tree import DecisionTreeRegressor # algoritmo Árvore de Regressão
from sklearn.svm import SVR # algoritmo SVM

In [36]:
# Configurações
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [37]:
dataset_part1 = r'https://raw.githubusercontent.com/rafaelpuyau/PUC-Rio/main/fipe_cars_part1.csv'
dataset_part2 = r'https://raw.githubusercontent.com/rafaelpuyau/PUC-Rio/main/fipe_cars_part2.csv'
dataset_part3 = r'https://raw.githubusercontent.com/rafaelpuyau/PUC-Rio/main/fipe_cars_part3.csv'
dataset_part4 = r'https://raw.githubusercontent.com/rafaelpuyau/PUC-Rio/main/fipe_cars_part4.csv'

df = pd.DataFrame()

for i in range(1, 5):
  df_aux = pd.read_csv(globals()[f'dataset_part{i}'])
  df = pd.concat([df, df_aux])

In [39]:
df.shape

(599007, 11)

In [135]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 599007 entries, 0 to 149751
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   year_of_reference   599007 non-null  int64  
 1   month_of_reference  599007 non-null  object 
 2   fipe_code           599007 non-null  object 
 3   authentication      599007 non-null  object 
 4   brand               599007 non-null  object 
 5   model               599007 non-null  object 
 6   fuel                599007 non-null  object 
 7   gear                599007 non-null  object 
 8   engine_size         599007 non-null  float64
 9   year_model          599007 non-null  int64  
 10  avg_price_brl       599007 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 54.8+ MB


In [136]:
print(df.isna().sum())

year_of_reference     0
month_of_reference    0
fipe_code             0
authentication        0
brand                 0
model                 0
fuel                  0
gear                  0
engine_size           0
year_model            0
avg_price_brl         0
dtype: int64

In [40]:
# Preparação dos dados

# Separação em bases de treino e teste (holdout)

# As colunas fipe_code, authentication, brand e model serão removidas pois não ajudarão no modelo
X =  df.drop(['fipe_code', 'authentication', 'model', 'avg_price_brl'], axis='columns') # atributos

y =  df['avg_price_brl'] # classe (target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7) # faz a divisão

# Buscando as colunas que não são numéricas
colunas_categoricas = X_train.select_dtypes(include=[object]).columns
print(colunas_categoricas)

# Criando o objeto que vai transformar as colunas categóricas em diversas colunas numéricas
ohe = OneHotEncoder(sparse=False, drop='first')

# Transformando as variáveis categóricas em numéricas
# Codificar as variáveis categóricas
# Os algoritmos entendem apenas números! Assim, devemos converter as variáveis
# categóricas de alguma forma. Para isso, utilizarei o _OneHotEncoding_, uma forma
# simples de transformar categorias não ordinais em números

for col_cat in colunas_categoricas:
  colunas_ohe = ohe.fit_transform(X_train[[col_cat]])
  colunas_ohe_teste = ohe.transform(X_test[[col_cat]])

  categorias_ohe = ohe.categories_[0][1:]

  for indice, nome_categoria in enumerate(categorias_ohe):
    df_categorias = pd.DataFrame(data=colunas_ohe[:, indice], columns=[nome_categoria])
    df_categorias_teste = pd.DataFrame(data=colunas_ohe_teste[:, indice], columns=[nome_categoria])

    X_train[nome_categoria] = colunas_ohe[:, indice]
    X_test[nome_categoria] = colunas_ohe_teste[:, indice]

  X_train = X_train.drop(col_cat, axis='columns')
  X_test = X_test.drop(col_cat, axis='columns')

# Criando os folds para a validação cruzada
num_particoes = 10 # número de folds da validação cruzada
kfold = KFold(n_splits=num_particoes, shuffle=True, random_state=7) # faz o particionamento em 10 folds

Index(['month_of_reference', 'brand', 'fuel', 'gear'], dtype='object')


In [ ]:
# Modelagem

# Definindo uma seed global para esta célula de código
np.random.seed(7)

# Listas para armazenar os modelos, os resultados e os nomes dos modelos
models = []
results = []
names = []

# Preparando os modelos e adicionando-os em uma lista
models.append(('LR', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVM', SVR()))

# Avaliando um modelo por vez
for name, model in models:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
  results.append(cv_results)
  names.append(name)
  # imprime MSE, desvio padrão do MSE e RMSE dos 10 resultados da validação cruzada
  msg = "%s: MSE %0.2f (%0.2f) - RMSE %0.2f" % (name, abs(cv_results.mean()), cv_results.std(), np.sqrt(abs(cv_results.mean())))
  print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure()
fig.suptitle('Comparação do MSE dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

LR: MSE 25764831023.93 (1732510609.90) - RMSE 160514.27
Ridge: MSE 25765015680.31 (1736170802.94) - RMSE 160514.85
Lasso: MSE 25769516078.00 (1733320020.86) - RMSE 160528.86


In [ ]:
# Criando um modelo com todo o conjunto de treino
model = LinearRegression()
model.fit(X_train, y_train)

# Fazendo as predições com o conjunto de teste
predictions = model.predict(X_test)

# Estimando o MSE e o RMSE no conjunto de teste
mse = mean_squared_error(y_test, predictions)
print("MSE %0.2f" % mse)
print("RMSE %0.2f" % np.sqrt(abs(mse)))

# Visão Computacional

## Instalar a biblioteca Ultralytics

In [ ]:
!pip install ultralytics==8.0.0 -q

In [ ]:
from IPython.display import Image
import torch

In [ ]:
torch.cuda.is_available()

## Detecção de objetos com YOLOv8

Vamos baixar duas imagens de exemplo do meu Google Drive para o Google Colab Notebook. Essas imagens mostrarão como o YOLOv8 funciona na detecção de objetos em imagens.

Vamos baixar a imagem de exemplo do Google Drive. Para isso, utilize o link direto de download e execute os comandos abaixo:

In [ ]:
!gdown "https://drive.google.com/uc?id=1YQB6NpKfQD6T9OAq8fDc_gL2vvvdiB5q"
!gdown "https://drive.google.com/uc?id=1pmTPY7_g_gg6Npqkas3KXL57vzqYhhEu"

## Executar Detecção de Objetos em uma Imagem de Exemplo

Agora que temos a imagem de exemplo baixada no Google Colab Notebook. Usaremos o YOLOv8 para executar a detecção de objetos na imagem. Para isso, siga os passos abaixo:

Primeiro, execute o seguinte comando na célula abaixo para carregar o modelo YOLOv8 pré-treinado e realizar a detecção de objetos na imagem de exemplo:

In [ ]:
!yolo task=detect mode=predict model=yolov8s.pt source='/content/foto1.jpg'

Após a execução bem-sucedida do comando, o resultado da detecção de objetos será salvo no diretório "runs/detect/exp". Para visualizar a imagem com os objetos detectados, execute o seguinte comando no bloco de notas:

In [ ]:
Image('/content/runs/detect/predict/foto1.jpg', width=4128)

O comando acima exibirá a imagem "da minha família" com os objetos detectados pelo modelo YOLOv8.

In [ ]:
# Outro exemplo
Image('/content/runs/detect/predict/foto2.jpg', width=2322)

## Para salvar as informações das caixas delimitadoras, defina save_txt=True

Se desejar salvar as informações das caixas delimitadoras (bounding boxes) detectadas pelo modelo YOLOv8, como as coordenadas e as classes dos objetos detectados, siga os passos abaixo:

Execute o seguinte comando no bloco de notas, adicionando o argumento --save-txt:

In [ ]:
!yolo task=detect mode=predict model=yolov8s.pt source='/content/foto1.jpg' save_txt=True

In [ ]:
# Outro exemplo
!yolo task=detect mode=predict model=yolov8s.pt source='/content/foto2.jpg' save_txt=True

## Display the Output Image

In [ ]:
Image('/content/runs/detect/predict2/foto1.jpg')

## Para salvar o "Objeto Recortado" defina save_crop=True

Se desejar salvar os objetos recortados (crop objects) detectados pelo modelo YOLOv8, siga os passos abaixo:

In [ ]:
!yolo task=detect mode=predict model=yolov8s.pt source='/content/foto1.jpg' save_crop=True

In [ ]:
# Outro exemplo
!yolo task=detect mode=predict model=yolov8s.pt source='/content/foto2.jpg' save_crop=True

Após a execução bem-sucedida do comando, os objetos recortados serão salvos no diretório "runs/detect/exp/crops". Para visualizar os arquivos de imagem dos objetos recortados, você pode usar o seguinte código no bloco de notas:

In [ ]:
Image('/content/runs/detect/predict3/crops/person/foto1.jpg')

In [ ]:
Image('/content/runs/detect/predict3/crops/person/foto2.jpg')

## Image Segmentation

In [ ]:
!yolo task=segment mode=predict model=yolov8s-seg.pt source='/content/foto1.jpg'

Para visualizar a imagem com a _Segmentation_

In [ ]:
Image('/content/runs/segment/predict/foto1.jpg')